In [2]:
import pandas as pd
import numpy as np

# Variables d'ensemble

In [15]:
sequence_size = 1

In [97]:
batch_size=10

# Import donnée

In [98]:
data_batch=pd.read_csv('../data/train_1M.csv',nrows=batch_size)

In [99]:
data_batch

,Unnamed: 0,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,90286654,0,1917429896,7900,0,0,0,1,NaN,NaN
1,1,90286655,26511,1917429896,7876,0,1,2,0,21000.0,False
2,2,90286656,55312,1917429896,175,0,2,0,0,21000.0,False
3,3,90286657,102436,1917429896,1278,0,3,0,0,23000.0,False
4,4,90286658,188454,1917429896,2063,0,4,3,0,45000.0,False
5,5,90286659,188454,1917429896,2065,0,4,2,1,45000.0,False
6,6,90286660,188454,1917429896,2064,0,4,2,0,45000.0,False
7,7,90286661,264296,1917429896,3365,0,5,0,0,26666.0,False
8,8,90286662,264296,1917429896,3364,0,5,1,1,26666.0,False
9,9,90286663,264296,1917429896,3363,0,5,2,0,26666.0,False


# Création des bases

In [111]:
data_sequences={}

In [112]:
data_qstats=pd.DataFrame(columns=['question_id','qstats_answered_correctly','qstats_count'])
data_qstats.set_index('question_id',drop=True,inplace=True)
data_qstats

,qstats_answered_correctly,qstats_count
question_id,,


# Fonctions

In [108]:
def user_sequences_update(row, data_qstats, data_sequences, sequence_size):
    '''Crée ou met à jour la séquence d'un utilisateur'''
    user=row['user_id']
    content=row['content_id']
    if row['content_type_id']==1:# TODO : coder le comportement de cette fonction quand elle arrive sur une lecture
        pass
    
    if user in data_sequences.keys():
        user_data=data_sequences[user]
        
        new_correctly_answered_cumcount = user_data.iloc[-1].correctly_answered_cumcount + user_data.iloc[-1].answered_correctly
        new_uncorrectly_answered_cumcount=user_data.iloc[-1].uncorrectly_answered_cumcount + 1- user_data.iloc[-1].answered_correctly
        new_user_avg_score_cum=new_correctly_answered_cumcount/(new_correctly_answered_cumcount + new_uncorrectly_answered_cumcount)

    else:
        user_data=None
        
        new_correctly_answered_cumcount=0
        new_uncorrectly_answered_cumcount=0
        new_user_avg_score_cum=0        
        
    new_line= pd.DataFrame({'answered_correctly':row['answered_correctly'],
                         'correctly_answered_cumcount':new_correctly_answered_cumcount,
                         'uncorrectly_answered_cumcount':new_uncorrectly_answered_cumcount,
                         'user_avg_score_cum':new_user_avg_score_cum,
                         'qstats_answered_correctly':data_qstats.loc[content].qstats_answered_correctly,
                                       },
                          index=[-1])
    user_data=pd.concat([user_data,new_line],axis=0,ignore_index=True)
    
    data_sequences[user] = user_data.iloc[-sequence_size:]
    return data_sequences

In [80]:
def qstats_update(row, data_qstats):
    user=row['user_id']
    question=row['content_id']
    
    if question in data_qstats.index:
        question_stats=data_qstats.loc[question]
        
        new_question_count=question_stats.qstats_count+1
        new_question_answered_correctly_count = question_stats.qstats_count * question_stats.qstats_answered_correctly + row['answered_correctly']
        new_question_answered_correctly_mean = new_question_answered_correctly_count / new_question_count
        
    else:
        new_question_count=1
        new_question_answered_correctly_mean=row['answered_correctly']
        
    data_qstats.loc[question]={'qstats_answered_correctly':new_question_answered_correctly_mean,
                                   'qstats_count':new_question_count,
                                  }
    return data_qstats

In [100]:
def preprocess_data(data_batch,
                    data_qstats,
                    data_sequences={},
                    sequence_size=1,
                    update_qstats=True,
                    return_data=False):
    '''Pour un batch de nouvelles données, met à jour la table des dernières sequences pour chaque individu
    et si option activée, met également à jour la table des qstats
    '''
    for index, row in data_batch.iterrows():
        #print(row.row_id)
        user=row['user_id']
        content=row['content_id']
        if row['content_type_id']==1:
            pass # TODO : code the features engineering for lectures. Pour le moment on skip
        
        else:
            
# Si l'option updata_sqtats est activée, on met à jour la table data_qstats
            if update_qstats == True:
                
                data_qstats=qstats_update(row, data_qstats)
            
# Si on vient de remplir q_stats pour la question OU si la question est déjà connue, 
# on peut mettre à ajouter la ligne courante aux sequences
            if update_qstats == True or content in data_qstats.question_id.values:
        
                data_sequences= user_sequences_update(row, data_qstats, data_sequences, sequence_size)
                
# Si la question est nouvelle ET on a pas mis l'option update_qstats, on lève une erreur car il manque la donnée qstats
            else:
                print(f'ERROR : unknown question n°{content}. Set update_qstats == True to create lines for new questions or check the completeness of data_qstats')

#TODO : retourner le df complet avec les features si option activée
    if return_data==False:
        return data_qstats, data_sequences
                

# Main loop

In [113]:
data_qstats, data_sequences = preprocess_data(data_batch,
                    data_qstats,
                    data_sequences={},
                    sequence_size=10,
                    update_qstats=True,
                    return_data=False)

90286654
90286655
90286656
90286657
90286658
90286659
90286660
90286661
90286662
90286663


In [114]:
data_qstats

,qstats_answered_correctly,qstats_count
question_id,,
7900,1,1
7876,0,1
175,0,1
1278,0,1
2063,0,1
2065,1,1
2064,0,1
3365,0,1
3364,1,1


In [116]:
data_sequences[1917429896]

,answered_correctly,correctly_answered_cumcount,uncorrectly_answered_cumcount,user_avg_score_cum,qstats_answered_correctly
0,1,0.0,0.0,0.000000,1
1,0,1.0,0.0,1.000000,0
2,0,1.0,1.0,0.500000,0
3,0,1.0,2.0,0.333333,0
4,0,1.0,3.0,0.250000,0
5,1,1.0,4.0,0.200000,1
6,0,2.0,4.0,0.333333,0
7,0,2.0,5.0,0.285714,0
8,1,2.0,6.0,0.250000,1
9,0,3.0,6.0,0.333333,0


# Essais en vrac

In [29]:
dat=pd.DataFrame({'machin':[2,3],'bidule':[4,5]})
3 in dat.bidule.values

False

In [85]:
dat

,bidule
machin,
2,999
3,5
8,9
12,10


In [34]:
dat=dat.append({'machin':8,'bidule':9},ignore_index=True)
dat

,machin,bidule
0,2,4
1,3,5
2,8,9


In [41]:
fut=None

In [42]:
dat

,machin,bidule
0,2,4
1,3,5
2,8,9


In [43]:
pd.concat([fut,dat],axis=0)

,machin,bidule
0,2,4
1,3,5
2,8,9


In [45]:
truc=pd.concat([dat,dat],axis=0)

In [48]:
truc.iloc[-4:]

,machin,bidule
2,8,9
0,2,4
1,3,5
2,8,9


In [49]:
truc

,machin,bidule
0,2,4
1,3,5
2,8,9
0,2,4
1,3,5
2,8,9


In [50]:
truc.iloc[-50:]

,machin,bidule
0,2,4
1,3,5
2,8,9
0,2,4
1,3,5
2,8,9


In [53]:
truc.set_index('machin', drop=True,inplace=True)

In [56]:
dat.set_index('machin', drop=True,inplace=True)

In [59]:
dat.loc[2].bidule=999

In [60]:
dat

,bidule
machin,
2,999
3,5
8,9


In [69]:
5 in dat.index

False

In [73]:
dat.loc[12]={'bidule':10}

In [74]:
dat

,bidule
machin,
2,999
3,5
8,9
12,10


In [91]:
dico={4:3,5:3}

In [94]:
8 in dico.keys()

False

In [105]:
pd.DataFrame({'neuneu':8,'zou':7},index=[0])

,neuneu,zou
0,8,7
